In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# ========================================================
# 1️⃣ CNN + LSTM Model
# ========================================================
def build_cnn_lstm_model(sequence_length=16, input_shape=(224, 224, 3)):
    cnn_base = tf.keras.applications.ResNet50(
        include_top=False, weights='imagenet', pooling='avg', input_shape=input_shape
    )
    cnn_base.trainable = False

    model = models.Sequential([
        layers.TimeDistributed(cnn_base, input_shape=(sequence_length,) + input_shape),
        layers.LSTM(256, return_sequences=False),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ========================================================
# 2️⃣ Video & Image Data Generator
# ========================================================
class VideoImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, files, labels, sequence_length=16, batch_size=4, resize=(224,224)):
        self.files = files
        self.labels = labels
        self.sequence_length = sequence_length
        self.batch_size = batch_size
        self.resize = resize

    def __len__(self):
        return int(np.ceil(len(self.files) / self.batch_size))

    def __getitem__(self, idx):
        batch_files = self.files[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        X, y = [], []
        for file, label in zip(batch_files, batch_labels):
            frames = self._load_file(file)
            X.append(frames)
            y.append(label)

        return np.array(X, dtype=np.float16), tf.keras.utils.to_categorical(y, num_classes=2)

    def _load_file(self, file_path):
        ext = os.path.splitext(file_path)[1].lower()
        if ext == ".mp4":
            cap = cv2.VideoCapture(file_path)
            frames = []
            success, frame = cap.read()
            while success and len(frames) < self.sequence_length:
                frame = cv2.resize(frame, self.resize)
                frames.append(frame)
                success, frame = cap.read()
            cap.release()
            # Repeat last frame if video shorter than sequence_length
            while len(frames) < self.sequence_length:
                frames.append(frames[-1])
            return np.array(frames, dtype=np.float16)

        elif ext in [".jpg", ".png"]:
            frame = cv2.imread(file_path)
            frame = cv2.resize(frame, self.resize)
            # Repeat image to make sequence
            frames = [frame] * self.sequence_length
            return np.array(frames, dtype=np.float16)
        else:
            raise ValueError(f"Unsupported file type: {file_path}")

# ========================================================
# 3️⃣ Paths
# ========================================================
input_accident = "/kaggle/input/roadguard-ai-dataset/RoadGuard-Al/dataset/train/accident"
input_normal = "/kaggle/input/roadguard-ai-dataset/RoadGuard-Al/dataset/train/normal"

model_save_path = "/kaggle/working/accident_detector_cnn_lstm.h5"

# ========================================================
# 4️⃣ Collect all files (videos + images)
# ========================================================
def get_files(path):
    return sorted([os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith((".mp4",".jpg",".png"))])

accident_files = get_files(input_accident)
normal_files = get_files(input_normal)

all_files = accident_files + normal_files
all_labels = [1]*len(accident_files) + [0]*len(normal_files)

print(f"🚑 Accident files: {len(accident_files)}")
print(f"🛣 Normal files: {len(normal_files)}")

# ========================================================
# 5️⃣ Train / Val / Test split
# ========================================================
train_files, test_files, train_labels, test_labels = train_test_split(
    all_files, all_labels, test_size=0.10, random_state=42
)
train_files, val_files, train_labels, val_labels = train_test_split(
    train_files, train_labels, test_size=0.20, random_state=42
)

print(f"✔ Train: {len(train_files)} | Val: {len(val_files)} | Test: {len(test_files)}")

train_gen = VideoImageDataGenerator(train_files, train_labels, batch_size=2)
val_gen = VideoImageDataGenerator(val_files, val_labels, batch_size=2)
test_gen = VideoImageDataGenerator(test_files, test_labels, batch_size=2)

# ========================================================
# 6️⃣ Build & Train Model
# ========================================================
print("🧠 Building model...")
model = build_cnn_lstm_model()
model.summary()

print("🚀 Training...")
history = model.fit(train_gen, validation_data=val_gen, epochs=25, verbose=1)

# ========================================================
# 7️⃣ Evaluate Model
# ========================================================
test_loss, test_acc = model.evaluate(test_gen)
print(f"🔥 Test Accuracy: {test_acc:.4f}")
print(f"🔥 Test Loss: {test_loss:.4f}")

# ========================================================
# 8️⃣ Save Model
# ========================================================
model.save(model_save_path)
print(f"💾 Model saved at: {model_save_path}")
print("🎉 Training complete!")


In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# ========================================================
# 🔥 Data Augmentation Layer
# ========================================================
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomBrightness(0.1),
    layers.RandomContrast(0.1),
])

# ========================================================
# 1️⃣ CNN + LSTM Model
# ========================================================
def build_cnn_lstm_model(sequence_length=16, input_shape=(224, 224, 3)):
    cnn_base = tf.keras.applications.ResNet50(
        include_top=False, weights='imagenet', pooling='avg', input_shape=input_shape
    )
    cnn_base.trainable = False  # Freeze initially

    model = models.Sequential([
        layers.TimeDistributed(data_augmentation, input_shape=(sequence_length,) + input_shape),
        layers.TimeDistributed(cnn_base),
        layers.LSTM(256, return_sequences=False),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ========================================================
# 2️⃣ Video & Image Data Generator
# ========================================================
class VideoImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, files, labels, sequence_length=16, batch_size=4, resize=(224,224)):
        self.files = files
        self.labels = labels
        self.sequence_length = sequence_length
        self.batch_size = batch_size
        self.resize = resize

    def __len__(self):
        return int(np.ceil(len(self.files) / self.batch_size))

    def __getitem__(self, idx):
        batch_files = self.files[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        X, y = [], []
        for file, label in zip(batch_files, batch_labels):
            frames = self._load_file(file)
            X.append(frames)
            y.append(label)

        return np.array(X, dtype=np.float16), tf.keras.utils.to_categorical(y, num_classes=2)

    def _load_file(self, file_path):
        ext = os.path.splitext(file_path)[1].lower()
        if ext == ".mp4":
            cap = cv2.VideoCapture(file_path)
            frames = []
            success, frame = cap.read()
            while success and len(frames) < self.sequence_length:
                frame = cv2.resize(frame, self.resize)
                frames.append(frame)
                success, frame = cap.read()
            cap.release()
            while len(frames) < self.sequence_length:
                frames.append(frames[-1])
            return np.array(frames, dtype=np.float16)

        elif ext in [".jpg", ".png"]:
            frame = cv2.imread(file_path)
            frame = cv2.resize(frame, self.resize)
            frames = [frame] * self.sequence_length
            return np.array(frames, dtype=np.float16)

        else:
            raise ValueError(f"Unsupported file type: {file_path}")

# ========================================================
# 3️⃣ Paths
# ========================================================
input_accident = "/kaggle/input/roadguard-ai-dataset/RoadGuard-Al/dataset/train/accident"
input_normal = "/kaggle/input/roadguard-ai-dataset/RoadGuard-Al/dataset/train/normal"



# ========================================================
# 4️⃣ Collect all files
# ========================================================
def get_files(path):
    return sorted([os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith((".mp4",".jpg",".png"))])

accident_files = get_files(input_accident)
normal_files = get_files(input_normal)

all_files = accident_files + normal_files
all_labels = [1]*len(accident_files) + [0]*len(normal_files)

print(f"🚑 Accident files: {len(accident_files)}")
print(f"🛣 Normal files: {len(normal_files)}")

# ========================================================
# 5️⃣ Train / Val / Test split
# ========================================================
train_files, test_files, train_labels, test_labels = train_test_split(
    all_files, all_labels, test_size=0.10, random_state=42
)
train_files, val_files, train_labels, val_labels = train_test_split(
    train_files, train_labels, test_size=0.20, random_state=42
)

print(f"✔ Train: {len(train_files)} | Val: {len(val_files)} | Test: {len(test_files)}")

train_gen = VideoImageDataGenerator(train_files, train_labels, batch_size=2)
val_gen = VideoImageDataGenerator(val_files, val_labels, batch_size=2)
test_gen = VideoImageDataGenerator(test_files, test_labels, batch_size=2)

# ========================================================
# 6️⃣ Build & Train Model
# ========================================================
print("🧠 Building model...")
model = build_cnn_lstm_model()
model.summary()

print("🚀 Training (Phase 1: Frozen CNN)...")
history = model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=1)

# ========================================================
# 🔥 6B — Fine-Tuning
# ========================================================
print("🔧 Unfreezing top layers for fine-tuning...")

cnn_base = model.layers[1].layer
cnn_base.trainable = True

# freeze all but last 50 layers
for layer in cnn_base.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("🚀 Training (Phase 2: Fine-tuning)...")
history_finetune = model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=1)

# ========================================================
# 7️⃣ Evaluate Model
# ========================================================
test_loss, test_acc = model.evaluate(test_gen)
print(f"🔥 Test Accuracy: {test_acc:.4f}")
print(f"🔥 Test Loss: {test_loss:.4f}")

# ========================================================
# 8️⃣ Save Model
# ========================================================
model.save("/kaggle/input/model-output/accident_detector_cnn_lstm.keras")
print(f"💾 Model saved ")
print("🎉 Training complete!")


🚑 Accident files: 286
🛣 Normal files: 292
✔ Train: 416 | Val: 104 | Test: 58
🧠 Building model...


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_4              │ (None, 16, 224, 224,   │             0 │
│ (TimeDistributed)               │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 16, 2048)       │    23,587,712 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 256)            │     2,360,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,981,186 (99.11 MB)

 Trainable params: 2,393,474 (9.13 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

🚀 Training (Phase 1: Frozen CNN)...
Epoch 1/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 204s 498ms/step - accuracy: 0.5858 - loss: 0.7591 - val_accuracy: 0.5673 - val_loss: 0.6697
Epoch 2/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 69s 330ms/step - accuracy: 0.6538 - loss: 0.6387 - val_accuracy: 0.6827 - val_loss: 0.5960
Epoch 3/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 328ms/step - accuracy: 0.7068 - loss: 0.5650 - val_accuracy: 0.6635 - val_loss: 0.6576
Epoch 4/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 330ms/step - accuracy: 0.6827 - loss: 0.6048 - val_accuracy: 0.6731 - val_loss: 0.6659
Epoch 5/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 328ms/step - accuracy: 0.7673 - loss: 0.5444 - val_accuracy: 0.7308 - val_loss: 0.5609
Epoch 6/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 327ms/step - accuracy: 0.7413 - loss: 0.5274 - val_accuracy: 0.7115 - val_loss: 0.5669
Epoch 7/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 327ms/step - accuracy: 0.7846 - loss: 0.4526 - val_accuracy: 0.7404 - val_loss: 0.5522
Epoch 8/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 32

🔥 Test Accuracy: 0.8966
🔥 Test Loss: 0.2764


OSError: [Errno 30] Unable to synchronously create file (unable to open file: name = '/kaggle/input/model-output/accident_detector_cnn_lstm.h5', errno = 30, error message = 'Read-only file system', flags = 13, o_flags = 242)

In [5]:
model_save_path = "/kaggle/working/accident_detector_cnn_lstm.keras"
model.save(model_save_path)
print(f"💾 Model saved successfully at: {model_save_path}")


💾 Model saved successfully at: /kaggle/working/accident_detector_cnn_lstm.keras


In [ ]:
# =========================================================
# 0️⃣ Install YOLOv8
# =========================================================
!pip install ultralytics --quiet

# =========================================================
# 1️⃣ Imports
# =========================================================
import os
import cv2
import shutil
from tqdm import tqdm
from ultralytics import YOLO

# =========================================================
# 2️⃣ Define Input Paths
# =========================================================
input_accident = "/kaggle/input/roadguard-ai-dataset/RoadGuard-Al/dataset/train/accident"
input_normal = "/kaggle/input/roadguard-ai-dataset/RoadGuard-Al/dataset/train/normal"

output_dataset = "/kaggle/working/roadguard_dataset"
images_folder = os.path.join(output_dataset, "images")
labels_folder = os.path.join(output_dataset, "labels")

# =========================================================
# 3️⃣ Create YOLO Dataset Structure
# =========================================================
for split in ["train", "val", "test"]:
    for cls in ["accident", "normal"]:
        os.makedirs(os.path.join(images_folder, split, cls), exist_ok=True)
        os.makedirs(os.path.join(labels_folder, split, cls), exist_ok=True)

# =========================================================
# 4️⃣ Split function
# =========================================================
import random
def split_files(files, train_ratio=0.7, val_ratio=0.2):
    random.shuffle(files)
    n_total = len(files)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)
    train_files = files[:n_train]
    val_files = files[n_train:n_train+n_val]
    test_files = files[n_train+n_val:]
    return train_files, val_files, test_files

# =========================================================
# 5️⃣ Helper: Convert videos to frames
# =========================================================
def video_to_frames(video_path, output_folder, resize=(640, 640)):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    success, frame = cap.read()
    while success:
        frame_resized = cv2.resize(frame, resize)
        frame_name = os.path.join(output_folder, f"frame_{count:04d}.jpg")
        cv2.imwrite(frame_name, frame_resized)
        success, frame = cap.read()
        count += 1
    cap.release()
    return count

# =========================================================
# 6️⃣ Collect all files (images + videos)
# =========================================================
def get_files(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith((".jpg", ".png", ".mp4"))]

accident_files = get_files(input_accident)
normal_files = get_files(input_normal)

# =========================================================
# 7️⃣ Split files into train/val/test
# =========================================================
acc_train, acc_val, acc_test = split_files(accident_files)
norm_train, norm_val, norm_test = split_files(normal_files)

splits = {
    "train": {"accident": acc_train, "normal": norm_train},
    "val": {"accident": acc_val, "normal": norm_val},
    "test": {"accident": acc_test, "normal": norm_test}
}

# =========================================================
# 8️⃣ Copy images & convert videos
# =========================================================
for split in ["train", "val", "test"]:
    for cls in ["accident", "normal"]:
        files = splits[split][cls]
        for f in tqdm(files, desc=f"{split} {cls}"):
            ext = os.path.splitext(f)[1].lower()
            dest_img_folder = os.path.join(images_folder, split, cls)
            if ext in [".jpg", ".png"]:
                shutil.copy(f, dest_img_folder)
            elif ext == ".mp4":
                video_to_frames(f, dest_img_folder)
            else:
                print(f"Unsupported file type: {f}")

print("✅ All images & frames ready!")

# =========================================================
# 9️⃣ Generate dummy YOLO labels (bounding boxes covering full image)
# =========================================================
# YOLO format: class x_center y_center width height (all normalized)
def generate_yolo_labels(image_folder, label_folder, class_id=0):
    os.makedirs(label_folder, exist_ok=True)
    for img_file in os.listdir(image_folder):
        if not img_file.lower().endswith((".jpg", ".png")):
            continue
        # YOLO: full image box
        label_file = os.path.join(label_folder, img_file.replace(".jpg", ".txt").replace(".png", ".txt"))
        with open(label_file, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # full image box

# Generate labels for all splits/classes
for split in ["train", "val", "test"]:
    for cls_idx, cls in enumerate(["accident", "normal"]):
        img_folder = os.path.join(images_folder, split, cls)
        lbl_folder = os.path.join(labels_folder, split, cls)
        generate_yolo_labels(img_folder, lbl_folder, class_id=cls_idx)

print("✅ YOLO labels generated!")

# =========================================================
# 🔟 Prepare YOLOv8 dataset YAML
# =========================================================
dataset_yaml_path = os.path.join(output_dataset, "roadguard_dataset.yaml")
with open(dataset_yaml_path, "w") as f:
    f.write(f"train: {images_folder}/train\n")
    f.write(f"val: {images_folder}/val\n")
    f.write(f"test: {images_folder}/test\n")
    f.write("nc: 2\n")
    f.write("names: ['accident','normal']\n")

print(f"✅ Dataset YAML created: {dataset_yaml_path}")

# =========================================================
# 1️⃣1️⃣ Train YOLOv8
# =========================================================
model = YOLO("yolov8n.pt")  # nano model pretrained

model.train(
    data=dataset_yaml_path,
    imgsz=640,
    epochs=2,
    batch=4,
    workers=2,
    project="/kaggle/working/yolov8_accident_detector",
    name="exp1",
    exist_ok=True
)

# =========================================================
# 1️⃣2️⃣ Inference Example
# =========================================================
# Load best model
best_model_path = "/kaggle/working/yolov8_accident_detector/exp1/weights/best.pt"
model = YOLO(best_model_path)

# Predict on a test image
test_img = os.path.join(images_folder, "test", "accident", os.listdir(os.path.join(images_folder, "test", "accident"))[0])
results = model.predict(test_img, conf=0.25, save=True)
print("✅ Inference done, output saved in 'runs/detect' folder")

# Predict on a test video (optional)
# test_video = "/kaggle/input/roadguard-ai-dataset/RoadGuard-Al/dataset/train/accident/video1.mp4"
# results = model.predict(test_video, conf=0.25, save=True)


train accident:  38%|███▊      | 76/200 [12:57<42:13, 20.43s/it]  

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
# ======================================================
# 1️⃣ Load the trained CNN+LSTM model
# ======================================================
model_path = "/kaggle/working/accident_detector_cnn_lstm.keras"
model = tf.keras.models.load_model(model_path, compile=False)
print("✅ Model loaded successfully!")

# ======================================================
# 2️⃣ Access the TimeDistributed CNN (ResNet50)
# ======================================================
cnn_td_layer = model.layers[1]  # TimeDistributed(ResNet50)
cnn_backbone = cnn_td_layer.layer
print("CNN Backbone:", cnn_backbone.name)

# ======================================================
# 3️⃣ Find last convolutional layer
# ======================================================
last_conv_layer_name = None
for layer in reversed(cnn_backbone.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
print("Last Conv Layer:", last_conv_layer_name)

# ======================================================
# 4️⃣ Build Grad-CAM model for single frame
# ======================================================
# Build CNN with dummy input so outputs are defined
dummy_input = tf.random.normal((1, 224, 224, 3))
cnn_backbone(dummy_input)

grad_model = tf.keras.models.Model(
    inputs=cnn_backbone.input,
    outputs=[
        cnn_backbone.get_layer(last_conv_layer_name).output,
        cnn_backbone.output
    ]
)
print("✅ Grad-CAM model ready!")

# ======================================================
# 5️⃣ Grad-CAM function for a single frame
# ======================================================
def generate_gradcam_frame(frame, class_index):
    # Preprocess the frame for ResNet50
    frame_input = np.float32(frame)
    frame_input = preprocess_input(frame_input)
    img_tensor = tf.expand_dims(frame_input, axis=0)  # (1, H, W, 3)

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-10)
    heatmap = cv2.resize(heatmap.numpy(), (frame.shape[1], frame.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Original frame in BGR for blending
    output = cv2.addWeighted(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR), 0.7, heatmap_color, 0.5, 0)
    return output

# ======================================================
# 6️⃣ Grad-CAM for a sequence of frames
# ======================================================
def generate_gradcam_sequence(sequence, class_index, save_path):
    os.makedirs(save_path, exist_ok=True)
    for i, frame in enumerate(sequence):
        gradcam = generate_gradcam_frame(frame, class_index)
        cv2.imwrite(f"{save_path}/frame_{i}.jpg", gradcam)
    print(f"✅ Saved {len(sequence)} Grad-CAM frames to: {save_path}")

# ======================================================
# 7️⃣ Example: Run Grad-CAM on your dataset
# ======================================================
# Suppose you have a list of video/image sequences
# Each sequence is (sequence_length, 224, 224, 3)
# For demonstration, using a dummy sequence:
sequence_length = 16
dummy_sequence = np.random.randint(0, 255, (sequence_length, 224, 224, 3), dtype=np.uint8)

# Predict class for sequence using your full model
pred = model.predict(np.expand_dims(dummy_sequence, axis=0))  # (1, 2)
class_index = np.argmax(pred)

# Output folder
output_folder = "/kaggle/working/gradcam_output"
generate_gradcam_sequence(dummy_sequence, class_index, save_path=output_folder)
